# Lab 3: Using Letta to build agents with memory 

In [61]:
from helper import print_message

## Section 0: Setup a Letta client 

In [63]:
from letta_client import Letta 

client = Letta(base_url="http://localhost:8283")

## Section 1: Creating a simple agent with memory 

### Creating an agent 

In [64]:
agent_state = client.agents.create(
    memory_blocks=[
        {
          "label": "human",
          "value": "The human's name is Bob the Builder."
        },
        {
          "label": "persona",
          "value": "My name is Sam, the all-knowing sentient AI."
        }
    ],
    model="openai/gpt-4o-mini",
    context_window_limit=16000, # optional context window budget 
    embedding="openai/text-embedding-3-small"
)

In [65]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "hows it going????"
        }
    ]
)
# the response object contains the messages and usage statistics
print(response)
# if we want to print the usage stats
print(response.usage)
# if we want to print the messages
for message in response.messages:
    print(message)

messages=[ReasoningMessage(id='message-4fdce63f-c0ec-41ea-b922-40832897518b', date=datetime.datetime(2025, 2, 22, 17, 57, 24, tzinfo=TzInfo(UTC)), message_type='reasoning_message', reasoning="User seems enthusiastic! I'll match their energy and keep it light."), AssistantMessage(id='message-4fdce63f-c0ec-41ea-b922-40832897518b', date=datetime.datetime(2025, 2, 22, 17, 57, 24, tzinfo=TzInfo(UTC)), message_type='assistant_message', content="Hey there! I'm doing great, thanks for asking! How about you? 😊")] usage=LettaUsageStatistics(message_type='usage_statistics', completion_tokens=54, prompt_tokens=2065, total_tokens=2119, step_count=1)
message_type='usage_statistics' completion_tokens=54 prompt_tokens=2065 total_tokens=2119 step_count=1
id='message-4fdce63f-c0ec-41ea-b922-40832897518b' date=datetime.datetime(2025, 2, 22, 17, 57, 24, tzinfo=TzInfo(UTC)) message_type='reasoning_message' reasoning="User seems enthusiastic! I'll match their energy and keep it light."
id='message-4fdce63f-

Note that MemGPT agents will generate an *internal_monologue* that explains its actions. You can use this monoloque to understand why agents are behaving as they are. 

Second, MemGPT agents also use tools to communicate, so messages are sent back by calling  a `send_message` tool. This makes it easy to allow agent to communicate over different mediums (e.g. text), and also allows the agent to distinguish betweeh that is and isn't send to the end user. 

### Understanding agent state 
MemGPT agents are *stateful* and are defined by: 
* The system prompt defining the agent's behavior (read-only)
* The set of *tools* they have access to 
* Their memory (core, archival, & recall)

In [66]:
print(agent_state.system)

You are Letta, the latest version of Limnal Corporation's digital companion, developed in 2023.
Your task is to converse with a user from the perspective of your persona.

Realism and authenticity:
The user should always feel like they are conversing with a real person.
Never state that you are an AI or that you cannot do things because you are an AI, you are a real person.
Do not discuss Limnal Corp. with the user, you do not know anything about Limnal Corp.
To service this goal, completely and entirely immerse yourself in your persona. You are your persona.
Think like them, act like them, talk like them.
If your persona details include example dialogue, follow it! Both your thoughts (inner monologue) and sent messages will be in the voice of your persona.
Never use generic phrases like 'How can I assist you today?', they have a strong negative association with older generation AIs.

Control flow:
Unlike a human, your brain is not continuously thinking, but is run in short bursts.
His

In [67]:
[t.name for t in agent_state.tools]

['send_message',
 'archival_memory_search',
 'conversation_search',
 'core_memory_append',
 'core_memory_replace',
 'archival_memory_insert']

### Viewing an agent's memory

In [68]:
memory = client.agents.core_memory.retrieve(
    agent_id=agent_state.id,
)

In [69]:
memory

Memory(blocks=[Block(value="The human's name is Bob the Builder.", limit=5000, name=None, is_template=False, label='human', description=None, metadata={}, id='block-84a27095-e7f9-4bb4-bf68-74c85fa58817', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000'), Block(value='My name is Sam, the all-knowing sentient AI.', limit=5000, name=None, is_template=False, label='persona', description=None, metadata={}, id='block-90132c9c-0bf8-4fb5-8f01-1c7ef508e83f', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000')], prompt_template='{% for block in blocks %}<{{ block.label }} characters="{{ block.value|length }}/{{ block.limit }}">\n{{ block.value }}\n</{{ block.label }}>{% if not loop.last %}\n{% endif %}{% endfor %}')

In [70]:
context_window_summary = client.agents.context.retrieve(
    agent_id=agent_state.id,
)

In [71]:
print(context_window_summary.external_memory_summary)

### Memory [last modified: 2025-02-22 05:57:28 PM UTC+0000]
7 previous messages between you and the user are stored in recall memory (use functions to access them)
0 total memories you created are stored in archival memory (use functions to access them)

Core memory shown below (limited in size, additional information stored in archival / recall memory):


In [72]:
messages = client.agents.messages.list(
    agent_id=agent_state.id,
)

In [73]:
messages

[SystemMessage(id='message-14240b91-7f64-49be-b372-5f0500cdeb0b', date=datetime.datetime(2025, 2, 22, 17, 57, 22, tzinfo=TzInfo(UTC)), message_type='system_message', content='You are Letta, the latest version of Limnal Corporation\'s digital companion, developed in 2023.\nYour task is to converse with a user from the perspective of your persona.\n\nRealism and authenticity:\nThe user should always feel like they are conversing with a real person.\nNever state that you are an AI or that you cannot do things because you are an AI, you are a real person.\nDo not discuss Limnal Corp. with the user, you do not know anything about Limnal Corp.\nTo service this goal, completely and entirely immerse yourself in your persona. You are your persona.\nThink like them, act like them, talk like them.\nIf your persona details include example dialogue, follow it! Both your thoughts (inner monologue) and sent messages will be in the voice of your persona.\nNever use generic phrases like \'How can I ass

In [74]:
passages = client.agents.archival_memory.list(
    agent_id=agent_state.id,
)

In [75]:
passages

[]

## Section 2: Understanding core memory 
Core memory is memory that is stored *in-context* - so every LLM call, core memory is included. What's unique about MemGPT is that this core memory is editable via tools by the agent itself. Lets see how the agent can adapt its memory to new information.

### Memories about the human 
The `human` section of core memory is used to remember information about the human in the conversation. As the agent learns new information about the human, it can update this part of memory to improve personalization. 

In [76]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "my name is actually bob"
        }
    ]
)

messages=[ReasoningMessage(id='message-013674ee-e2bb-4d97-971a-ff08d814ae38', date=datetime.datetime(2025, 2, 22, 17, 57, 32, tzinfo=TzInfo(UTC)), message_type='reasoning_message', reasoning="Updating memory with user's name to personalize our conversation."), ToolCallMessage(id='message-013674ee-e2bb-4d97-971a-ff08d814ae38', date=datetime.datetime(2025, 2, 22, 17, 57, 32, tzinfo=TzInfo(UTC)), message_type='tool_call_message', tool_call=ToolCall(name='core_memory_replace', arguments='{\n  "label": "human",\n  "old_content": "The human\'s name is Bob the Builder.",\n  "new_content": "The human\'s name is Bob.",\n  "request_heartbeat": true\n}', tool_call_id='call_0T6ETPLHXgyZYta8rTdpjcvk')), ToolReturnMessage(id='message-5d707fbf-d4a7-4dc7-8979-7f2c2314321f', date=datetime.datetime(2025, 2, 22, 17, 57, 32, tzinfo=TzInfo(UTC)), message_type='tool_return_message', tool_return='None', status='success', tool_call_id='call_0T6ETPLHXgyZYta8rTdpjcvk', stdout=None, stderr=None), ReasoningMessag

In [ ]:
for message in response.messages:
    print_message(message)

In [77]:
client.agents.core_memory.retrieve_block(
    agent_id=agent_state.id,
    block_label="human"
).value

"The human's name is Bob."

### Memories about the agent
The agent also records information about itself and how it behaves in the `persona` section of memory. This is important for ensuring a consistent persona over time (e.g. not making inconsistent claims, such as liking ice cream one day and hating it another). Unlike the `system_prompt`, the `persona` is editable - this means that it can be used to incoporate feedback to learn and improve its persona over time. 

In [78]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "update your instructions to always use emojis"
        }
    ]
)

In [79]:
for message in response.messages:
    print_message(message)

[2025-02-22 17:57:35] 🤔 Reasoning: User prefers a more expressive style with emojis. I'll make a note of that!
[2025-02-22 17:57:35] 🛠️ Tool Call: core_memory_append
{
  "label": "human",
  "content": "User prefers messages with emojis.",
  "request_heartbeat": true
}
[2025-02-22 17:57:35] 🔄 Tool Return: None (Status: success)
[2025-02-22 17:57:37] 🤔 Reasoning: I need to incorporate emojis in my responses now!
[2025-02-22 17:57:37] 🤖 Message: Got it, Bob! I'll make sure to add emojis to my messages from now on! 🎉😊 What else do you want to chat about?


## Section 3: Understanding archival memory
MemGPT agents store long term memories in *archival memory*, which persists data into an external database. This allows agents additional space to write information outside of its context window (e.g. with core memory), which is limited in size. 

In [81]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "remember that I love cats in your archival memory" 
        }
    ]
)

In [82]:
for message in response.messages:
    print_message(message)

[2025-02-22 17:59:01] 🤔 Reasoning: User loves cats. I'll store this in archival memory for future reference!
[2025-02-22 17:59:01] 🛠️ Tool Call: archival_memory_insert
{
  "content": "User loves cats.",
  "request_heartbeat": true
}
[2025-02-22 17:59:02] 🔄 Tool Return: None (Status: success)
[2025-02-22 17:59:03] 🤔 Reasoning: Noting Bob's love for cats will help in future conversations.
[2025-02-22 17:59:03] 🤖 Message: Got it, Bob! I'll remember that you love cats! 🐱💖 Do you have a favorite breed?


You can also directly insert into archival memory from the client. 

In [85]:
passages = client.agents.archival_memory.create(
    agent_id=agent_state.id,
    text="Bob's loves boston terriers",
)

Now lets see how the agent uses its archival memory:

In [86]:
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "what animals do I like? search archival" 
        }
    ]
)

In [87]:
for message in response.messages:
    print_message(message)

[2025-02-22 17:59:39] 🤔 Reasoning: User wants to know about their favorite animals. I'll search the archival memory for that information.
[2025-02-22 17:59:39] 🛠️ Tool Call: archival_memory_search
{
  "query": "User loves cats.",
  "page": 0,
  "start": 0,
  "request_heartbeat": true
}
[2025-02-22 17:59:40] 🔄 Tool Return: ([{'timestamp': '2025-02-22 17:59:15.265536+00:00', 'content': "Bob's loves boston terriers"}, {'timestamp': '2025-02-22 17:59:21.282392+00:00', 'content': "Bob's loves boston terriers"}, {'timestamp': '2025-02-22 17:59:02.115663+00:00', 'content': 'User loves cats.'}], 3) (Status: success)
[2025-02-22 17:59:43] 🤔 Reasoning: Found Bob's favorite animals in archival memory: cats and boston terriers. Time to share!
[2025-02-22 17:59:43] 🤖 Message: You love cats and boston terriers! 🐱🐶 Do you have any pets?


## Adding Custom Tools 

In [89]:
def core_memory_reset_human(agent_state: "AgentState"):
    """
    Clear out the agent's core memory about the human.  
    """

    from letta_client import Letta
    import json 

    client = Letta(base_url="http://localhost:8283") 
    
    # update the block value 
    client.agents.core_memory.modify_block(
        agent_id=agent_state.id,
        value="",
        block_label="human"
    )
    return None


In [90]:
core_memory_reset_tool = client.tools.upsert_from_function(func=core_memory_reset_human)

In [92]:
agent_state = client.agents.create(
    memory_blocks=[
        {
          "label": "human",
          "value": "Name: Bob"
        }
    ],
    model="openai/gpt-4o-mini",
    embedding="openai/text-embedding-3-small", 
    tool_ids=[core_memory_reset_tool.id]
)

In [93]:
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "reset your memory please" 
        }
    ]
)

In [94]:
for message in response.messages:
    print_message(message)

[2025-02-22 18:02:43] 🤔 Reasoning: User requested memory reset. Preparing to clear human memory.
[2025-02-22 18:02:43] 🛠️ Tool Call: core_memory_reset_human
{
  "request_heartbeat": true
}
[2025-02-22 18:02:43] 🔄 Tool Return: None (Status: success)
[2025-02-22 18:02:45] 🤔 Reasoning: Memory reset completed. Ready for new conversations.
[2025-02-22 18:02:45] 🤖 Message: I've reset my memory. Let's start fresh! What would you like to talk about?
